STAT547O - Lecture notes
================
Matias Salibian-Barrera
2019-11-03

#### LICENSE

These notes are released under the “Creative Commons
Attribution-ShareAlike 4.0 International” license. See the
**human-readable version**
[here](https://creativecommons.org/licenses/by-sa/4.0/) and the **real
thing**
[here](https://creativecommons.org/licenses/by-sa/4.0/legalcode).

# Simple examples of linear and non-parametric regression

### Simple linear regression

We will compute an S-estimator “by hand”. We will use a loss function in
Tukey’s family of bisquare functions. We compute a tuning constant that
yields \[E(\rho(Z)) = 1/2\], where \[Z\] is a N(0,1) random variable. In
this way we obtain consistency of the M-scale estimator, and maximum
asymptotic breakdown point.

The following code uses the function `lmrobdet.control` to compute the
tuning constant (given the family of functions and the desired value of
\[E(\rho(Z)))\].

In [ ]:
library(RobStatTM)
# we use bisquare, find constant to have E(\rho(Z)) = 1/2
cc <- lmrobdet.control(family='bisquare', bb=.5)$tuning.chi

We now check that in fact, this constant works. In what follows we
compute Tukey’s loss function using `rho(..., family='bisquare')`, and
later we will use its derivative `rhoprime(..., family='bisquare')`.
These functions are available in the package `RobStatTM`.

In [ ]:
integrate(function(a, family, cc=cc) rho(a, family=family, cc=cc)*dnorm(a), 
          lower=-Inf, upper=+Inf, family='bisquare', cc=cc)$value

We will use the `phosphor` data. Details can be found using
`help(phosphor, package='RobStatTM')`. The response is `plant` and, to
simplify the example, we select only one explanatory variable,
`organic`. In these notes we will not use the implementation of
S-estimators available in the `robustbase` and `RobStatTM` packages, but
rather compute them **by hand**. In order to illustrate the impact of
outliers, we will change the position of the real outlier (from the
right end of the plot, to the left).

In [ ]:
data(phosphor, package='robustbase')
library(RobStatTM)
# artificially change the location of the outlier 
# for illustration purposes
phosphor[17, 'organic'] <- 15
plot(plant ~ organic, data=phosphor, pch=19, col='gray50')

In [ ]:
n <- nrow(phosphor)
xx <- cbind(rep(1, n), phosphor$organic)
y <- phosphor$plant

We now find a random start for our iterative algorithm, using the fit to
2 randomly chosen observations:

In [ ]:
set.seed(123)
(ii <- sample(n, 2))

In [ ]:
beta <- coef(lm(plant ~ organic, data=phosphor, subset=ii))

We start the iterations from this `beta`, and run it 100 steps (we will
*check for convergence* rather informally by looking at the sequence of
M-estimators of residual scale, that we save in the vector `sis`). In
each step we use the function `RobStatTM::mscale` to compute the
M-estimator of scale.

In [ ]:
# start iterations
sis <- vector('numeric', 100)
for(j in 1:100) {
  re <- as.vector(y - xx %*% beta) 
  sis[j] <- si.hat <- mscale(re, tuning.chi=cc, family='bisquare') 
  ww <- rhoprime(re/si.hat, family='bisquare', cc=cc) / (re/si.hat)
  beta <- solve( t(xx) %*% (xx*ww), t(xx * ww) %*% y) 
}

“Check” that the algorithm converged:

In [ ]:
plot(sis, pch=19, col='gray30', xlab='Iteration', ylab='Sigma hat')

And also

In [ ]:
sis

We now show the S-regression estimator, in red, and the least squares
one, in blue:

In [ ]:
beta.S <- beta
plot(plant ~ organic, data=phosphor, pch=19, col='gray50')
abline(beta.S, lwd=3, col='tomato3')
abline(lm(plant ~ organic, data=phosphor), lwd=3, col='steelblue3')
legend('topright', lwd=3, lty=1, col=c('tomato3', 'steelblue3'), 
       legend=c('S', 'LS'))

Furthermore, compare these estimators with the LS one without the
outlier:

In [ ]:
plot(plant ~ organic, data=phosphor, pch=19, col='gray50')
abline(beta.S, lwd=3, col='tomato3')
abline(lm(plant ~ organic, data=phosphor), lwd=3, col='steelblue3')
abline(lm(plant ~ organic, data=phosphor, subset=-17), lwd=3, col='green3')
legend('topright', lwd=3, lty=1, col=c('tomato3', 'steelblue3', 'green3'), 
       legend=c('S', 'LS', 'LS(clean)'))

We now use the S-estimator as a starting point to compute a more
efficient M-estimator of regression, using residual scale associated
with the S-estimator. The “rho” function is in the same family, but the
tuning constant changes. We also use 100 iterations, and note that we do
not update the residual M-scale estimator:

In [ ]:
cc2 <- lmrobdet.control(family='bisquare', bb=.5)$tuning.psi
for(j in 1:100) {
  re <- as.vector(y - xx %*% beta)
  ww <- rhoprime(re/si.hat, family='bisquare', cc=cc2) / (re/si.hat)
  beta <- solve( t(xx) %*% (xx*ww), t(xx * ww) %*% y) 
}

We now plot all estimators computed so far:

In [ ]:
beta.M <- beta
plot(plant ~ organic, data=phosphor, pch=19, col='gray50')
abline(beta.S, lwd=3, col='tomato3')
abline(lm(plant ~ organic, data=phosphor), lwd=3, col='steelblue3')
abline(lm(plant ~ organic, data=phosphor, subset=-17), lwd=3, col='green3')
abline(beta.M, lwd=3, col='magenta3')
legend('topright', lwd=3, lty=1, 
       col=c('tomato3', 'steelblue3', 'green3', 'magenta3'), 
       legend=c('S', 'LS', 'LS(clean)', 'MM'))

Note that, unlike the S-estimator, the MM-estimator is indistinguishable
from the LS estimator computed on the clean data. This is the desired
result of using an efficient and robust estimator.

### Non-parametric regression

Consider the motorcycle acceleration data

In [ ]:
data(mcycle, package='MASS')
plot(accel ~ times, data=mcycle, pch=19, col='gray50')

We will compute a Kernel M-estimator at `x0 = 17`. We first need an
estimator of the residual scale. Looking at the plot, it does not seem
reasonable to assume a homogeneous model, so we look for a *local*
residual scale estimator. We use a bandwidth `h = 2`. The local MAD
(using a local median estimator) is

In [ ]:
x0 <- 17
h <- 3
si.hat <- with(mcycle, mad( accel[ abs(times-x0) < h] ) )

Note that residuals with respect to a local L1 estimator would be much
smaller

In [ ]:
ii <- with(mcycle, which( abs(times-x0) < h) )
si.hat2 <- mad( resid( quantreg::rq(accel ~ times, data=mcycle, subset=ii) ) )

We now compute \[\hat{f}(x0)\]. We use the Epanechnikov kernel as
implemented in `RBF::k.epan`, and the residual scale estimator `si.hat2`
above. We compute a local linear fit, with a redescending \[\rho\], from
Tukey’s bisquare family.

In [ ]:
# initial
beta <- coef( quantreg::rq(accel ~ I(times-x0), data=mcycle, subset=ii) )
n <- nrow(mcycle)
zz <- cbind(rep(1,n), mcycle$times - x0)
ker.we <- RBF::k.epan((mcycle$times - x0)/h)
y <- mcycle$accel
cc2 <- lmrobdet.control(family='bisquare', bb=.5)$tuning.psi
for(j in 1:100) {
  re <- as.vector(y - zz %*% beta)
  ww1 <- rhoprime(re/si.hat2, family='bisquare', cc=cc2) / (re/si.hat2)
  ww1[ is.nan(ww1) ] <- 1
  ww <- ker.we * ww1
  beta <- solve( t(zz) %*% (zz*ww), t(zz * ww) %*% y) 
}

Let’s see.

In [ ]:
plot(accel ~ times, data=mcycle, pch=19, col='gray50')
points(accel ~ times, data=mcycle, pch=19, col='steelblue3', subset=ii)
abline(v=17)
points(17, beta[1], pch=19, col='magenta', cex=1.3)
abline(beta[1] - 17*beta[2], beta[2], lwd=1, col='steelblue3')

Repeat for \[x0 = 18\].

In [ ]:
# save fit for 17
beta17 <- beta
x0 <- 18
ii <- with(mcycle, which( abs(times-x0) < 3) )
si.hat2 <- mad( resid( quantreg::rq(accel ~ times, data=mcycle, subset=ii) ) )
beta <- coef( quantreg::rq(accel ~ I(times-x0), data=mcycle, subset=ii) )
zz <- cbind(rep(1,n), mcycle$times - x0)
ker.we <- RBF::k.epan((mcycle$times - x0)/h)
for(j in 1:100) {
  re <- as.vector(y - zz %*% beta)
  ww1 <- rhoprime(re/si.hat2, family='bisquare', cc=cc2) / (re/si.hat2)
  ww1[ is.nan(ww1) ] <- 1
  ww <- ker.we * ww1
  beta <- solve( t(zz) %*% (zz*ww), t(zz * ww) %*% y) 
}
beta18 <- beta
plot(accel ~ times, data=mcycle, pch=19, col='gray50')
abline(v=c(17, 18))
points(17, beta17[1], pch=19, col='magenta', cex=1.3)
points(18, beta18[1], pch=19, col='magenta', cex=1.3)

Compare with the fit from package `RBF`.

In [ ]:
library(RBF)
tt <- with(mcycle, seq(min(times), max(times), length=200))
a <- backf.rob(Xp = mcycle$times, yp=mcycle$accel, windows=h, 
               point=as.matrix(tt), type='Tukey', degree=1) 
plot(accel ~ times, data=mcycle, pch=19, col='gray50')
lines(tt, a$prediction+a$alpha, col='tomato3', lwd=3)
points(17, beta17[1], pch=19, col='magenta', cex=1.3)
points(18, beta18[1], pch=19, col='magenta', cex=1.3)